# Mapeo campos 

|Fuente | Sources |
|---|---|
|Nombre base para flora de Bogota | Taxa |
|Comentario de validación | Taxa.Comment |
|Para revisión | Taxa.CheckPriority 0 or 1 |
|Endemica | Taxa.Distribution |
|Estado de amenaza | RedListAssessments with Standard='JBB'
|Origen y status | Origins |
|Trabajo de No nativas Flora de Bogotá | Origins.Type???
|Determinador en fuente | Identifications.IdentifiedBy Merge with next field |
|Determinador Depurado | Identifications.IdentifiedBy |
|Calificador en fuente | Identifications.Certainty |
|Fecha_Determinacion en fuente | Identifications.Date |
|Fecha_Determinacion_(Year) | Identifications.Date |
|Fecha_Determinacion_(Month) | Identifications.Date |
|Fecha_Determinacion_(Day) | Identifications.Date |
|State | Areas |
|County | Areas |
|Localidad | Locations |
|Código de localidad georreferenciada | Locations.BD2020ID |
|Latitud_original | Locations.Latitude |
|Longitud_original | Locations.Longitud |
|Elev__minima | Locations.ElevationMin |
|Elev__maxima | Locations.ElevationMax |
|Latitud_decimal | Geocodings.InterpretedLat, Geocoder is DB_2020 |
|Longitud_decimal | Geocodings.InterpretedLon, Geocoder is DB_2020 |
|Latitud para trabajo de mapas | Geocodings.InterpretedLat, Geocoder is mapas_2020 |
|Longitud para trabajo de mapas | Geocodings.InterpretedLon, Geocoder is mapas_2020 |
|Extensión de georreferenciación | Geocodins.Uncertainty ???
|DATUM | Geocodings.Datum |
|Colector depurado | Occurrences.Collector |
|Colector principal | If no colector depurado, Occurrences.Collector, otherwise Occurrences.CollectorVerbatim  |
|Número de colector | Occurrences.CollectionNumber & Occurrences.CollectionNumberVerbatim |
|Fecha_Inicial_(Year) | Occurrences.DateInit, Occurrences.DateEnd |
|Fecha_Inicial_(Month) | Occurrences.DateInit, Occurrences.DateEnd |
|Fecha_Inicial_(day) | Occurrences.DateInit, Occurrences.DateEnd |
|Notas_Localidad_y_Habitat | Occurrences.Comment  |
|Fenologia | Occurrences.PhenoState |
|Descripcion ejemplar | Occurrences.Comment |
|Usos | Occurrences.Use |
|Nombres_comunes | Occurrences.CommonName |
|Comentarios generales | Occurrences.Comment |
|Referencia bibligráfica | Occurrence.Sources |
|Numero original en Base 2015 | Occurrence.DB2015ID |
|Origen para herbario | Specimens.Institution  |
|Número de identificación | Specimens.SpecimenCode |
|Fecha_Inicial | REMOVE |
|Notas de altura | REMOVE ???????
|Familia Revisada | REMOVE |
|Género revisado | REMOVE |
|Comentarios de determinación en fuente | REMOVE |
|Nivel taxonómico | REMOVE, info retrieved from TPRS |
|Familia en fuente | REMOVE  |
|Género en fuente | REMOVE  |
|Epíteto específico en fuente | REMOVE  |
|Autor del epíteto específico en fuente | REMOVE  |
|Epíteto infraespecífico | REMOVE  |
|Autor del infraespecífico en fuente | REMOVE  |
|Nombre científico en fuente | REMOVE  |
|Nombre validado | REMOVE  |
|Nivel taxonómico en fuente | REMOVE  |
|Herbario de proveniencia | REMOVE  |

In [53]:
from sqlalchemy import create_engine, text
import pandas as pd
import numpy as np

import requests
import json
import re
import time

import pykew.powo as powo
from pygbif import species, registry

import credentials

In [34]:
orifile = '../Datos/Flora_de_Bogota/Registros_2020/FlBogota2020.csv'
ori = pd.read_csv(orifile, dtype={'Elev__minima': str, 'Elev__maxima': str})

In [2]:
tnrs_url = 'https://tnrsapi.xyz/tnrs_api.php'

In [3]:
def tropicos(name, api_key):
	service = 'http://services.tropicos.org/name'
	r = requests.get(
		f'{service}/search/?apikey={api_key}&name={name}&format=json'
	)
	#s = requests.get(
	#	f'{service}/{r['NameId']}/highertaxa/?apikey={api_key}&name={name}&format=json'
	#)
	
	return r.json()


In [4]:
def powo_query(name):
	act_name = ''
	bits = re.split(r'\s+', name)

	if bits[0] == '×' or bits[1] == '×':
		act_name = ' '.join(bits[:3])
	else:
		act_name = ' '.join(bits[:2])
	
	r = powo.search(act_name)

	if r.size() == 1:
		results = [i for i in r]
		return (results[0]['name'], results[0]['author'], results[0]['rank'])
	
	elif r.size() > 1:
		return f'Multiples results with name {name}'
	
	else:
		return f'No results with name {name}'

### Si la consulta a TNRS ya fue realizada y los resultados guardados:

In [5]:
# Map between original name and corrected version by TNRS
taxa_map_file = 'tnrs_query.csv'
taxa_map = pd.read_csv(taxa_map_file)

### De lo contrario, aqui se realiza la consulta

In [ ]:
orinames = ori['Nombre base para flora de Bogota'].unique()
query = [[i, j] for i,j in enumerate(orinames) if pd.notna(j)]

r = requests.post(
	tnrs_url, 
	headers={"Content-Type": "application/json"},
	data=json.dumps({
		'data': query,
		'opts': {
			'sources': ['wcvp','wfo'],
			'class': 'wcvp',
			'mode': 'resolve',
			'matches' : 'best'
		}		
	})	
)

In [ ]:
taxa_map = pd.DataFrame.from_records(results)
taxa_map.to_csv('tnrs_query.csv', index=False)

In [ ]:
taxa_map.loc[taxa_map.Overall_score == '', 'Overall_score'] = 'NAN'
taxa_map['Overall_score'] = taxa_map.Overall_score.astype(np.float32)

# Aqui se continua la normalizacion de nombres

### Consulta a POWO de hibridos

Primero remover hibridos no publicados, solo se registran a nivel de determinacion

In [6]:
taxa_map = taxa_map.drop(
	index=taxa_map.loc[taxa_map.Name_submitted.str.contains(r'Salvia [\w\.\s]+ ×')].index
)

In [ ]:
for row in taxa_map.loc[taxa_map.Name_submitted.str.contains('×')].itertuples():
	#print(row.Name_submitted)
	po = powo_query(row.Name_submitted)
	if isinstance(po, tuple):
		taxa_map.loc[row.Index, 'Name_matched'] = po[0]
		taxa_map.loc[row.Index, 'Author_matched'] = po[1]
		taxa_map.loc[row.Index, 'Name_matched_rank'] = po[2].lower()
	else:
		print(po)
	time.sleep(1)

No results with name Glandularia × hortulanus
Multiples results with name Pelargonium × hortorum L.H.Bailey
Multiples results with name Abelia × grandiflora (Andre) Rehder
No results with name Polygala × dalmasiana
Multiples results with name Lavatera × clementii Cheek


In [8]:
# Manually correct names couldn't be matched easily by POWO,
# but there is a simple solution there 

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Pelargonium × hortorum')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = ('Pelargonium × hortorum', 'L.H.Bailey', 'species')

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Abelia × grandiflora')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = ('Abelia × grandiflora', '(Rovelli ex André) Rehder', 'species')

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Lavatera × clementii')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = ('Lavatera × clementii', 'Cheek', 'species')

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Polygala × dalmasiana')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = ('Polygala', np.nan, 'genus')

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Epidendrum chicoense')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = ('Epidendrum', np.nan, 'genus')

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Plantae')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = (np.nan, np.nan, np.nan)

# TNRS couldn't get appropriate response for the following

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Callianthe striata')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = ('Callianthe striata', '(G.F.Dicks. ex Lindl.) Donnell', 'species')

taxa_map.loc[
	(taxa_map.Name_submitted.str.startswith('Phlegmariurus acifolius')), 
	['Name_matched', 'Author_matched', 'Name_matched_rank']
	] = ('Phlegmariurus acifolius', '(Rolleri) B.Øllg.', 'species')


In [9]:
# Save eliminated binomials from the database, they probably are invalid or unpublished names
taxa_map.loc[(taxa_map.Overall_score < 0.9) & (taxa_map.Name_matched_rank == 'genus'),
	['Name_submitted', 'Name_matched', 'Name_matched_rank']
	].to_csv('binomios_eliminados.csv', index=False)

In [27]:
# Remove lichens: Ramalina, Usnea, Sticta, Candelariella
# Remove general indets: 'Pteridophyta', 'Magnoliophyta', 'Plantae', 'Tracheophyta'

taxa_map = taxa_map.drop(
	index=taxa_map[taxa_map.Name_submitted.isin([
		'Ramalina', 'Usnea', 'Sticta', 'Candelariella',
		'Pteridophyta', 'Magnoliophyta', 'Plantae', 'Tracheophyta'
	])].index
)

In [30]:
taxa_map.to_csv('taxa_map.csv', index=False)

### Obtener jerarquía taxonómica de World Flora Online

In [2]:
taxa_map = pd.read_csv('taxa_map.csv')

In [3]:
wfo_file = '../../Botanical_databases/world_flora_online/classification.csv'

In [4]:
datyp = {'localID':str, 'subfamily': str, 'tribe': str, 'subtribe': str, 
 	'subgenus':str, 'infraspecificEpithet': str, 'tplID': str}

In [5]:
wfo = pd.read_csv(wfo_file, sep='\t', encoding='latin', dtype=datyp)

In [8]:
taxa_in = taxa_map.groupby(['Name_matched','Author_matched']
	).size(
	).reset_index(
	).drop(columns=0
	).rename(columns={'Name_matched': 'Name', 'Author_matched':'Author'})

In [ ]:
# For some reason Passiflora tripartita is retrieved twice, one with the wrong authorship
taxa_in = taxa_in.drop(
	index=taxa_in[
		(taxa_in.Name == 'Passiflora tripartita') & 
		(taxa_in.Author == '(Juss.) Poir.')
	].index).reset_index()

In [21]:
def get_family(name):
	ret = np.nan
	if pd.isna(name): return np.nan
	bits = re.split(r'\s+', name)
	nname = bits[0]
	out = wfo.loc[(wfo.scientificName == nname) & (wfo.taxonRank == 'genus')]

	if out.shape[0] == 1: 
		ret =  out['family'].item()
	
	elif out.shape[0] > 1:

		if out[out['nomenclaturalStatus'] == 'Valid'].shape[0] == 1:
			ret =  out.loc[out['nomenclaturalStatus'] == 'Valid', 'family'].item()

		elif out[out['taxonomicStatus'] == 'Accepted'].shape[0] == 1:
			ret =  out.loc[out['taxonomicStatus'] == 'Accepted', 'family'].item()

		else:
			if len(bits) > 1: nname = f'{bits[0]} {bits[1]}'
			out = wfo.loc[(wfo.scientificName == nname)]
			fams = out.family.unique()
			fams = [x for x in fams if pd.notna(x)]

			if len(fams) == 1:
				ret =  fams[0]		
		
	elif out.shape[0] < 1 and re.search(r'\w+aceae$', nname):
		out = wfo.loc[(wfo.scientificName == nname) & (wfo.taxonRank == 'family')]
		fams = out.family.unique()
		fams = [x for x in fams if pd.notna(x)]

		if len(fams) == 1:
			ret =  fams[0]

	return ret


In [22]:
taxa_in['wfo_family'] = taxa_in.Name.apply(get_family)

In [23]:
# Some missing names in World Flora Online
taxa_in.loc[taxa_in.Name == 'Hydrophyllaceae', 'wfo_family'] = 'Hydrophyllaceae'
taxa_in.loc[taxa_in.Name == 'Stemmatella', 'wfo_family'] = 'Asteraceae'
taxa_in.loc[taxa_in.Name == 'Coniza', 'wfo_family'] = 'Asteraceae'
taxa_in.loc[taxa_in.Name == '× Agropogon lutosus', 'wfo_family'] = 'Poaceae'


In [24]:
famis = taxa_in.groupby('wfo_family'
	).size(
	).reset_index(
	)

In [25]:
def get_papa(name):
	ret = np.nan

	if pd.notna(name):
		out = wfo.loc[(wfo.scientificName == name), 'parentNameUsageID']
		th = [x for x in out if pd.notna(x)]
		
		if len(th) == 1:
			parid = th[0]
			ret = wfo.loc[(wfo.taxonID == parid), 'scientificName'].item()
	
	return ret

In [26]:
famis['wfo_order'] = famis.wfo_family.apply(get_papa)

In [27]:
famis.loc[famis.wfo_family == 'Hydrophyllaceae', 'wfo_order'] = 'Solanales'

In [28]:
famis['wfo_clade'] = famis.wfo_order.apply(get_papa)

In [29]:
taxa_in = taxa_in.merge(famis, 'left', 'wfo_family')

In [31]:
taxa_in.drop(columns=0).to_csv('taxa_in.csv', index=False)

### Verificación de nombres realizada y previamente guardada localmente

Abajo se formalizan las jerarquías dado el diseño de la tabla Taxa en la DB

In [140]:
taxa_in = pd.read_csv('taxa_in.csv')

In [32]:
taxa_in[['CheckPriority', 'Distribution', 'Parent']] = np.nan, np.nan, np.nan

In [35]:
oriname = 'Nombre base para flora de Bogota'

for row in taxa_in.itertuples():
	
	qu = (ori[oriname] == row.Name) & (ori.Endemica == 'SI')
	
	if ori.loc[qu].shape[0] > 0:
		taxa_in.loc[taxa_in.Name == row.Name, 'Distribution'] = 0

	qu = (ori[oriname] == row.Name) & (ori['Para revisión'] == 'SI')
	
	if ori.loc[qu].shape[0] > 0:
		taxa_in.loc[taxa_in.Name == row.Name, 'CheckPriority'] = 5

In [36]:
taxa_in = taxa_in.reset_index(drop=True)

In [37]:
last_id = taxa_in.index[-1]

In [38]:
for th in taxa_in.wfo_clade.unique():
	qu = taxa_in.loc[taxa_in.Name == th]
	if qu.shape[0] == 0:
		last_id += 1
		rec = {i:np.nan for i in taxa_in.columns}
		rec['Name'] = th
		taxa_in = pd.concat([
			taxa_in, 
			pd.DataFrame.from_records([rec], index=[last_id])
		])


In [39]:
for th in taxa_in.wfo_order[taxa_in.wfo_order.notna()].unique():
	qu = taxa_in.loc[taxa_in.Name == th]
	if qu.shape[0] == 0:
		last_id += 1
		papa = taxa_in.loc[taxa_in.wfo_order == th, 'wfo_clade'].iloc[0]
		papaid = taxa_in.loc[taxa_in.Name == papa].index[0]
		rec = {i:np.nan for i in taxa_in.columns}
		rec['Name'] = th
		rec['Parent'] = papaid
		taxa_in = pd.concat([
			taxa_in, 
			pd.DataFrame.from_records([rec], index=[last_id])
		])


In [40]:
for th in taxa_in.wfo_family[taxa_in.wfo_family.notna()].unique():
	qu = taxa_in.loc[taxa_in.Name == th]
	if qu.shape[0] == 0:
		last_id += 1
		papa = taxa_in.loc[taxa_in.wfo_family == th, 'wfo_order'].iloc[0]
		papaid = taxa_in.loc[taxa_in.Name == papa].index[0]
		rec = {i:np.nan for i in taxa_in.columns}
		rec['Name'] = th
		rec['Parent'] = papaid
		taxa_in = pd.concat([
			taxa_in, 
			pd.DataFrame.from_records([rec], index=[last_id])
		])
	else:
		papa = taxa_in.loc[taxa_in.wfo_family == th, 'wfo_order'].iloc[0]
		papaid = taxa_in.loc[taxa_in.Name == papa].index[0]
		taxa_in.loc[taxa_in.Name == th, 'Parent'] = papaid



In [41]:
no_gens = taxa_in.wfo_family.unique().tolist() + \
	taxa_in.wfo_order.unique().tolist() + \
	taxa_in.wfo_clade.unique().tolist() + ['×']

no_gens = [i for i in no_gens if pd.notna(i)]

In [42]:
gens = [x for x in taxa_in.Name.apply(lambda x: re.split(r'\s+', x)[0]).unique()]
gens = [x for x in gens if not x in no_gens]

In [43]:
for gn in gens:
	#print(gn)
	
	if pd.notna(gn):

		qu = taxa_in.loc[taxa_in.Name == gn]

		if qu.shape[0] == 0:
			last_id += 1
			papa = taxa_in.loc[taxa_in.Name.str.startswith(f'{gn} '), 'wfo_family'].iloc[0]
			#print(papa)
			papaid = taxa_in.loc[taxa_in.Name == papa].index[0]
			#print(papaid)
			rec = {i:np.nan for i in taxa_in.columns}
			rec['Name'] = gn
			rec['Parent'] = papaid
			taxa_in = pd.concat([
				taxa_in, 
				pd.DataFrame.from_records([rec], index=[last_id])
			])

		else:
			papa = taxa_in.loc[taxa_in.Name == gn, 'wfo_family'].iloc[0]
			#print(papa)
			papaid = taxa_in.loc[taxa_in.Name == papa].index[0]
			#print(papaid)
			taxa_in.loc[taxa_in.Name == gn, 'Parent'] = papaid



In [44]:
for sp in taxa_in.loc[
	taxa_in.Name.str.contains(r'^[\w\-]+\s+\w+') |
	taxa_in.Name.str.contains(r'\w+\s+×'),
	'Name'
	].unique():

	#print(sp)
	gn = re.split(r'\s+', sp)[0]
	#print(gn)
	qu = taxa_in.loc[taxa_in.Name == gn]
	papaid = taxa_in.loc[taxa_in.Name == gn].index[0]
	#print(papaid)
	taxa_in.loc[taxa_in.Name == sp, 'Parent'] = papaid

In [45]:
taxa_in.loc[taxa_in.Name == '× Agropogon lutosus', 'Parent'
	] = taxa_in.loc[taxa_in.Name == 'Andropogon'].index[0]

In [46]:
taxa_in['TaxonID'] = taxa_in.index

In [48]:
taxa_in.to_csv('taxa_in.csv', index=False)

In [49]:
conn_str = 'mysql+mysqlconnector://' + \
	f'{credentials.mysql['username']}:{credentials.mysql['password']}' + \
	'@localhost:3306/Mutis'
engine = create_engine(conn_str)
connection = engine.connect()

INFO:package: mysql.connector.plugins
INFO:plugin_name: caching_sha2_password
INFO:AUTHENTICATION_PLUGIN_CLASS: MySQLCachingSHA2PasswordAuthPlugin


In [54]:
connection.execute(text('SET FOREIGN_KEY_CHECKS = 0'))

In [63]:
taxa_in[['TaxonID', 'Name', 'Author', 'CheckPriority', 'Distribution', 'Parent']
	].to_sql('Taxa', connection, if_exists='replace', index=False, chunksize=10000)

-1

In [ ]:
# Multispecies hybrids

taxa_map.loc[taxa_map.Name_submitted.str.contains('×') & 
	taxa_map.Name_matched.notna() & 
	~taxa_map.Name_matched.str.contains(r'×',na=False)]

-------

In [64]:
df = pd.read_sql(sql='SELECT * FROM Taxa', con=connection)

In [66]:
df

,TaxonID,Name,Author,CheckPriority,Distribution,Parent
0,0,Aa colombiana,Schltr.,NaN,None,3414.0
1,1,Aa hartwegii,Garay,NaN,None,3414.0
2,2,Aa leucantha,(Rchb.f.) Schltr.,NaN,None,3414.0
3,3,Aa maderoi,Schltr.,NaN,None,3414.0
4,4,Aa paleacea,(Kunth) Rchb.f.,NaN,None,3414.0
...,...,...,...,...,...,...
4435,4435,Zea,None,NaN,None,3226.0
4436,4436,Zephyranthes,None,NaN,None,3233.0
4437,4437,Zingiber,None,NaN,None,3347.0
4438,4438,Zuloagaea,None,NaN,None,3226.0


In [67]:
connection.close()